## Simple overhead benchmark

CPython 3.6.3 and an i7 7700K were used for the timings

In [1]:
import dependency_manager as dym
import attr

container = dym.container

In [2]:
@dym.service
class Service1(object):
    pass


@dym.service
class Service2(object):
    def __init__(self, service1: Service1):
        self.service1 = service1
       
 
@dym.service
class Service3(object):
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2

  
@dym.service
class Service4(object):
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3

### Function call

In [3]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

%timeit f(container[Service1], container[Service2], container[Service3], container[Service4])

685 ns ± 6.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
f_injected = dym.inject(f)
%timeit f_injected()

1.79 µs ± 4.32 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
f_prepared = dym.injector.prepare(f)

%timeit f_prepared()

209 ns ± 2.89 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Object instantiation

In [6]:
class Obj:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(container[Service1], container[Service2], container[Service3], container[Service4])

908 ns ± 6.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
class ObjInjected:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    @dym.inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

2.07 µs ± 28.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
@attr.s
class ObjAttrs:
    s1: Service1 = dym.attrib()
    s2: Service2 = dym.attrib()
    s3: Service3 = dym.attrib()
    s4: Service4 = dym.attrib()
        
%timeit ObjAttrs()